**MODIS NDVI value**

In [ ]:
# Load MODIS dataset for the specified date range
modis = ee.ImageCollection("MODIS/006/MOD09GA") \
    .filterDate('2002-01-01', '2002-12-31') \
    .filterBounds(adm2)

# Load cropland mask (USGS GFSAD 1km dataset)
crop_mask = ee.Image("USGS/GFSAD1000_V1")

# Function to mask clouds
def mask_clouds(image):
    cloud_mask = image.select('state_1km').bitwiseAnd(3).eq(0)
    return image.updateMask(cloud_mask)

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
    return image.addBands(ndvi)

# Apply cloud masking and calculate NDVI
modis_ndvi = modis.map(mask_clouds).map(calculate_ndvi)

# Mask cropland areas
def mask_cropland(image):
    return image.updateMask(crop_mask)

modis_ndvi_cropland = modis_ndvi.map(mask_cropland)

# Add year and month metadata to each image
def add_date_properties(image):
    date = ee.Date(image.get('system:time_start'))
    year = date.get('year')
    month = date.get('month')
    return image.set({'year': year, 'month': month})

modis_ndvi_cropland = modis_ndvi_cropland.map(add_date_properties)

# Filter for images with less than 10% cloud cover
def add_cloud_percentage(image):
    cloud_pixels = image.select('state_1km').bitwiseAnd(3).neq(0).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=adm2.geometry(),
        scale=1000,
        maxPixels=1e13
    ).get('state_1km')
    total_pixels = image.select('state_1km').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=adm2.geometry(),
        scale=1000,
        maxPixels=1e13
    ).get('state_1km')
    cloud_percentage = ee.Number(cloud_pixels).divide(ee.Number(total_pixels)).multiply(100)
    return image.set('cloud_percentage', cloud_percentage)

modis_ndvi_cropland = modis_ndvi_cropland.map(add_cloud_percentage)
modis_filtered = modis_ndvi_cropland.filter(ee.Filter.lt('cloud_percentage', 10))


# Group NDVI by ADM2, year, and month
def process_year_month(year, month, image_collection, admin_boundaries):
    # Filter by year and month
    monthly_images = image_collection \
        .filter(ee.Filter.eq('year', year)) \
        .filter(ee.Filter.eq('month', month))

    # If there are no valid images, assign -9999 for the mean and median NDVI
    def assign_invalid():
        return admin_boundaries.map(lambda feature: feature.set({
            'year': year,
            'month': month,
            'ADM2_PT': feature.get('ADM2_PT'),
            'ADM2_PCODE': feature.get('ADM2_PCODE'),
            'Shape_Area': feature.get('Shape_Area'),
            'NDVI_mean': -9999,   # Placeholder for mean NDVI
            'NDVI_median': -9999  # Placeholder for median NDVI
        }))

    # If valid images exist, calculate the mean and median NDVI
    if monthly_images.size().getInfo() == 0:
        return assign_invalid()
    else:
        # Calculate mean NDVI
        mean_ndvi = monthly_images.select('NDVI').mean()
        # Calculate median NDVI
        median_ndvi = monthly_images.select('NDVI').median()

        # Reduce the mean and median over regions
        mean_reduced = mean_ndvi.reduceRegions(
            collection=admin_boundaries,
            reducer=ee.Reducer.mean(),
            scale=1000
        )
        median_reduced = median_ndvi.reduceRegions(
            collection=admin_boundaries,
            reducer=ee.Reducer.median(),
            scale=1000
        )

        # Combine the results into a single FeatureCollection
        return mean_reduced.map(lambda f: f.set({
            'NDVI_median': median_reduced.filter(
                ee.Filter.equals('ADM2_PCODE', f.get('ADM2_PCODE'))
            ).first().get('median'),
            'year': year,
            'month': month
        }))

def reduce_to_monthly_means(image_collection, admin_boundaries):
    months = ee.List.sequence(1, 12)  # Months 1 to 12
    years = ee.List.sequence(2002, 2002)  # Year 2002 only

    results = []
    for year in years.getInfo():
        for month in months.getInfo():
            results.append(process_year_month(year, month, image_collection, admin_boundaries))

    # Flatten the list of FeatureCollections into a single FeatureCollection
    return ee.FeatureCollection(results).flatten()

# Reduce NDVI data by ADM2 boundaries
monthly_ndvi = reduce_to_monthly_means(modis_filtered, adm2)

# Select only the requested fields
monthly_ndvi_cleaned = monthly_ndvi.map(lambda f: f.set({
    'NDVI_mean': f.get('mean'),
    'NDVI_median': f.get('NDVI_median'),
    'ADM2_PT': f.get('ADM2_PT'),
    'ADM2_PCODE': f.get('ADM2_PCODE'),
    'year': f.get('year'),
    'month': f.get('month'),
    'Shape_Area': f.get('Shape_Area')
}))

# Export results to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=monthly_ndvi_cleaned,
    description='V1_Mozambique_mean_medi_2002',
    fileFormat='CSV'
)
task.start()


**Fusion NDVI value**

In [ ]:
# Define spatial and temporal weights
modis_spatial_weight = 1 / 250  # MODIS spatial resolution (250m)
landsat_spatial_weight = 1 / 30  # Landsat spatial resolution (30m)

modis_temporal_weight = 1 / 8  # MODIS temporal resolution (8-day composites)
landsat_temporal_weight = 1 / 16  # Landsat temporal resolution (16-day intervals)

# Combine spatial and temporal weights
modis_combined_weight = modis_spatial_weight * modis_temporal_weight
landsat_combined_weight = landsat_spatial_weight * landsat_temporal_weight

# Normalize weights to sum to 1
total_weight = modis_combined_weight + landsat_combined_weight
modis_combined_weight /= total_weight
landsat_combined_weight /= total_weight

# Add a new column to the merged dataset for fused NDVI
merged_data['fused_NDVI'] = (
    modis_combined_weight * merged_data['MODIS_mean'] +
    landsat_combined_weight * merged_data['LANDSAT_mean']
)

**Fixed Fusion NDVI value**

In [ ]:
# Define fixed weights
modis_fixed_weight = 0.3  # Adjust this to 0.5, 0.7, etc.
landsat_fixed_weight = 0.7  # Adjust this to 0.5, 0.3, etc.

# Ensure weights sum to 1 (optional step to verify)
assert modis_fixed_weight + landsat_fixed_weight == 1, "Weights must sum to 1."

# Calculate fused NDVI using fixed weights
merged_data['fused_NDVI_M3L7'] = (
    modis_fixed_weight * merged_data['MODIS_mean'] +
    landsat_fixed_weight * merged_data['LANDSAT_mean']
)